<a href="https://colab.research.google.com/github/zainabbasi38/Chatbots/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU langchain-community langchain-google-genai langchain-pinecone langchain-text-splitters

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.0 MB/s eta 0:00:00


In [ ]:
!pip install -qU pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 4.9 MB/s eta 0:00:00


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata
api_key = userdata.get("GEMINI_API_KEY")

llm = ChatGoogleGenerativeAI(
    api_key= api_key,
    model = "gemini-1.5-flash"
)

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(google_api_key=api_key, model="models/embedding-001" )
vector = embeddings.embed_query("Hi")
vector

[0.0014760760823264718,
 -0.029935583472251892,
 -0.01783810183405876,
 -0.047277648001909256,
 0.027684327214956284,
 -0.00889522209763527,
 0.030663948506116867,
 -0.01426528301090002,
 0.003124301554635167,
 0.021755851805210114,
 0.056663304567337036,
 0.018048927187919617,
 -0.006203501485288143,
 -0.05175986513495445,
 -0.020865578204393387,
 0.006319219246506691,
 0.004397958982735872,
 0.00570949399843812,
 0.010415352880954742,
 -0.02124374359846115,
 0.04982486367225647,
 0.03187477961182594,
 -0.010323568247258663,
 -0.005915531888604164,
 0.013109584338963032,
 -0.04946610704064369,
 0.0284867063164711,
 -0.04388665407896042,
 -0.028827698901295662,
 0.06491994857788086,
 -0.08532701432704926,
 0.03090032935142517,
 -0.053116269409656525,
 -0.003776727942749858,
 0.05218495428562164,
 -0.08170139044523239,
 0.023761693388223648,
 0.028207849711179733,
 -0.013105872087180614,
 0.046341318637132645,
 0.00500134751200676,
 -0.0534532368183136,
 -0.05413735657930374,
 -0.033191

In [ ]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone , ServerlessSpec


import time
pinecone_api = userdata.get("PINECONE_API_KEY")
index_name = "rag1"  # change if desired

pc =Pinecone(api_key=pinecone_api)
pc.create_index(
    name=index_name,
    dimension=768,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)
index = pc.Index(index_name)

vector_store = PineconeVectorStore(embedding=embeddings , index=index)

In [ ]:

from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load documents
loader = TextLoader("/content/sample_data/document.txt")  # Replace with your file
documents = loader.load()

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)
print(docs)

RuntimeError: Error loading my name is zain

In [ ]:
!pip install tqdm

In [ ]:
from tqdm import tqdm
vector = []
for doc in tqdm(docs):
  vector = embeddings.embed_query(doc.page_content)
  id = str(hash(doc.page_content))
  # Include metadata during upsert
  index.upsert(vectors=[(id , vector, {"text": doc.page_content})])


NameError: name 'docs' is not defined

In [ ]:
from langchain.vectorstores import Pinecone

retriver = Pinecone(index=index , embedding=embeddings , text_key="text")

retriever  = vector_store.as_retriever()



In [ ]:
from langchain.chains import RetrievalQA

retrieval_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="refine",
    retriever=retriever,
    return_source_documents=False
)

In [ ]:

while True:
  user = input("Ask something: ")
  if user.lower() == "exit":
    break
  response = retrieval_chain.run(user)
  print(response)